In [1]:
import numpy as np
import pandas as pd
import random
import string
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
import math
import json
from nltk import tokenize
import collections
import re
import itertools
import nltk
from scipy.stats import mannwhitneyu

import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel,\
        GenericLikelihoodModelResults

from statsmodels.nonparametric.smoothers_lowess import lowess

from scipy.special import zeta
from scipy.stats import binom

import pickle
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import numpy as np
import numpy.random as rand
from scipy.special import zeta
from scipy.misc import derivative

import numpy as np
lg = np.log10

## Data processing

In [2]:
# Pre-processing without part of speech tags
def remove_punctuation(text):
    text = text.lower()
    chars_to_remove = "[\n]!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
    tr = str.maketrans(" ", " ", chars_to_remove)
    return text.translate(tr)


def preprocess(corpus, sent = True):
    if sent:
        corpus = tokenize.sent_tokenize(corpus)
        corpus = [remove_punctuation(sent).split() for sent in corpus]
    else:
        corpus = remove_punctuation(corpus).split()
    return corpus

In [3]:
# Pre-processing with part of speech tags
def part_of_speech(corpus):
    corpus = tokenize.sent_tokenize(corpus)
    chars_to_remove = "[\n]"
    tr = str.maketrans(" ", " ", chars_to_remove)
    chars_to_remove2 = "[\n]!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
    new_corp = []
    test = []

    for sent in corpus:
        sent = sent.translate(tr)
        words_sent = tokenize.word_tokenize(sent)
        sent_pos = nltk.pos_tag(words_sent)
        new_sent = []
        for (word, pos) in sent_pos:
            tr2 = str.maketrans("", "", chars_to_remove2)
            word = word.translate(tr2)
            if word:
                new_sent.append((word.lower(), pos))
        new_corp.append(new_sent)
    return new_corp

In [4]:
# Total preprocessing function for a corpus. Input can be one string (corpus),
# for which you put multi = False, or a list of several strings (corpora) that 
# you want to turn into one big corpus, for which you put multi = True.
# For PoS tags, put pos = True.
def make_file(corp, multi = False, sent = True, pos = False):
    if multi:
        corpus = ''
        for subcorp in corp:
            corpus += subcorp
    else:
        corpus = corp
        
    if pos:
        corpus = part_of_speech(corpus)
    
    else:
        corpus = preprocess(corpus, sent = sent)
    
    return corpus

### Subsampling

In [5]:
# Returns 2 lists of corpora, one from which the ranks will be calculated
# and one from which the frequencies will be calculated. Each corpus consists of
# a list of tokenized sentences.
# Input: corpus that is to be subsampled. Should be a list of tokenized sentences.
# k is the amount of tokens that each sampled corpus should contain,
# m is the amount of subcorpera you want for both the ranks and frequencies.
# Max: I would read Valentin's thesis for an explanation on subsampling
def subsampling(corpus, k = 1000000, m = 10, sent = True):
    n = len(corpus)
    
    sen_len = {}

    
    rank_corpera = []
    freq_corpera = []

    if sent:
        for i in range(m):
            used_rank = set()
            used_freq = set()
            rank_count = 0
            freq_count = 0
            rank_samples = []
            freq_samples = []

            while rank_count < k:
                index = np.random.randint(n)
                if index in used_rank:
                    continue

                rank_sample = corpus[index]
                len_sample = len(rank_sample)

                if len_sample == 0:
                    continue

                rank_samples += rank_sample
                rank_count += len_sample

                if len_sample not in sen_len and len_sample < 200:
                    sen_len[len_sample] = 1
                elif len_sample < 200:
                    sen_len[len_sample] += 1


                used_rank.add(index)

            while freq_count < k:
                index = np.random.randint(n)
                if index in used_freq:
                    continue
                freq_sample = corpus[index]
                len_sample = len(freq_sample)

                if len_sample == 0:
                    continue

                freq_samples += freq_sample
                freq_count += len_sample

                if len_sample not in sen_len and len_sample < 200:
                    sen_len[len_sample] = 1
                elif len_sample < 200:
                    sen_len[len_sample] += 1

                used_freq.add(index)

            rank_corpera.append(rank_samples)
            freq_corpera.append(freq_samples)
#                 rank_corpera.append([item for sublist in rank_samples for item in sublist])
#                 freq_corpera.append([item for sublist in freq_samples for item in sublist])


    else:
        for i in range(m):
            rank_samples = random.sample(corpus, k)
            freq_samples = random.sample(corpus, k)
            rank_corpera.append(rank_samples)
            freq_corpera.append(freq_samples)
    
#     return rank_corpera, freq_corpera, sen_len
    return rank_corpera, freq_corpera

### Calculating the frequencies

In [6]:
# Returns a dataframe of word frequencies for list of corpora,
# with each column corresponding to a different corpus.
# Input: list of corpora. Each corpus consists of a list of tokenized sentences.
def calculate_freqs(freq_sents):
    freq_dict = {}
    for i, corpus in enumerate(freq_sents):
        freq_dict[i] = collections.Counter(corpus)
        
    freqs_df = pd.DataFrame(freq_dict)
    freqs_df = freqs_df.fillna(0)
    
    return freqs_df

In [7]:
# Returns a dataframe with the mean frequency of each word across different corpora.
# Input: frequency dataframe
def mean_freqs(freqs_df):
    return(freqs_df.mean(axis=1))

### Calculating the ranks

In [8]:
# Returns a dataframe of word ranks for list of corpora,
# with each column corresponding to a different corpus.
# Input: list of corpora. Each corpus consists of a list of tokenized sentences.
def calculate_ranks(rank_sents):
    ranks_dicts = {}
    for i, corpus in enumerate(rank_sents):
        freqs = collections.Counter(corpus)
        ranks_dicts[i] = {w: r for r, (w, c) in enumerate(freqs.most_common(), 1)}
        
    ranks_df = pd.DataFrame(ranks_dicts)
    for column in ranks_df:
        min_rank = int(np.ceil(ranks_df[column].max() + 1))
        nan_rows = ranks_df[ranks_df[column].isnull()]
        num_nans = len(nan_rows)
        nan_ranks = list(range(min_rank, min_rank+num_nans))
        random.shuffle(nan_ranks)
        ranks_df.loc[ranks_df[column].isnull(), column] = nan_ranks

    return ranks_df

In [9]:
# Returns a dataframe with the mean rank of each word across different corpora.
# Input: rank dataframe
def mean_ranks(ranks_df):
    return ranks_df.mean(axis=1)

In [10]:
# Creates combined dataframe of ranks and frequencies
# Input: 2 lists (freq_sents and rank_sents) of corpora. Each corpus
# consists of a list of tokenized sentences. These lists are to be obtained form
# subsampling.
def ranks_freqs(freq_sents, rank_sents):
    freqs_df = calculate_freqs(freq_sents)
    freqs_df['Frequency'] = mean_freqs(freqs_df)
    ranks_df = calculate_ranks(rank_sents)
    ranks_df['Rank'] = mean_ranks(ranks_df)
    
    # Put mean ranks and freqs together and remove all words that
    # do not have both a rank and frequency (which happens when a word)
    # is only present in freq_sents and not in rank_sents or vice versa
    ranks_freqs_df = pd.concat([ranks_df, freqs_df], axis = 1)
    ranks_freqs_df = ranks_freqs_df.dropna()
#     ranks_freqs_df = ranks_freqs_df.loc[ranks_freqs_df['Frequency'] >=1]
    return ranks_freqs_df

In [11]:
def freq_of_freqs_hist_plot(freqs):
    freq_of_freqs = collections.Counter(freqs)
    plt.bar(list(freq_of_freqs.keys()), freq_of_freqs.values(), color='b')
    plt.show()
    return None

In [12]:
def freq_of_freqs_hexbin_plot(freqs):
    freq_of_freqs = collections.Counter(freqs)
    hexbin_plot(list(freq_of_freqs.keys()), freq_of_freqs.values())
    plt.show()
    return None

## Fano factor

In [13]:
def fano_factor(ranks_freqs_df, ylim = None):
    fano = []

    for index, row in ranks_freqs_df.iterrows():
        fano.append((np.cov(row[0:10], row[11:21])[0][1])/row['Rank'])
    ranks_freqs_df['Fano'] = fano
#     fano_plot(ranks_freqs_df, ylim=ylim)
    return ranks_freqs_df

### Estimating Zipf's law

We estimate Zipf's law using $f(w) \propto \frac{1}{r(w)^{\alpha}}$. In the following code block, the $\alpha$ is found using maximum likelihood estimation, for which the code was taken from https://stats.stackexchange.com/questions/6780/how-to-calculate-zipfs-law-coefficient-from-a-set-of-top-frequencies .

In [14]:
# MLE of Zipf's law parameters (alpha and beta)
class Mandelbrot(GenericLikelihoodModel):

    def __init__(self, frequencies, ranks, **kwargs):
        if not len(frequencies) == len(ranks):
            raise ValueError("NOT THE SAME NUMBER OF RANKS AND FREQS!")
        
        frequencies = np.asarray(frequencies)
        ranks = np.asarray(ranks)
        
        self.n_obs = np.sum(frequencies)
        
        super().__init__(endog=frequencies, exog=ranks, **kwargs)
        self.fit_result = None
    

    def prob(self, params, ranks=None, log=False):
        if ranks is None:
            ranks = self.exog
        
        alpha, beta = params
        if log:
            return -alpha*lg(beta+ranks) - lg(zeta(alpha, q=beta+1.))
        else:
            return ((beta + ranks)**(-alpha))/zeta(alpha, q=beta+1.)
    
    
    def loglike(self, params):
        rs = self.exog
        fs = self.endog
        alpha, beta = params
        
#        if alpha > 10 or beta > 20:
#            return -np.inf
        
#         if alpha < 1.0 or beta < 0.0:
#             return -np.inf
        
        # no need to calculate P(r) when observed f(r) was zero
        log_probs = -alpha*lg(beta+rs) - lg(zeta(alpha, q=beta+1.))
        log_probs = log_probs.reshape(-1, )
        return np.sum(fs * log_probs) - beta**5
    
    
    def register_fit(self, fit_result, overwrite=False):
        if not self.fit_result is None and not overwrite:
            raise ValueError("A fit result is already registered and overwrite=False!")
            
        self.fit_result = fit_result
        self.optim_params = fit_result.params
        self.pseudo_r_squared = self.pseudo_r_squared(self.optim_params)
        self.SE, self.SE_relative = fit_result.bse, fit_result.bse/self.optim_params
        self.BIC, self.BIC_relative = fit_result.bic,\
                            (-2*self.null_loglike())/fit_result.bic

        return self.optim_params
    
    def print_result(self, string=False):
        if self.fit_result is None:
            raise ValueError("Register a fitting result first!")

        def format_x(x):
            return float('{0:.3g}'.format(x))


        s = "="*50
        s += "\n" + "MANDELBROT"
        s += "\n" + "  Optimal Parameters " + str(tuple(map(format_x, self.optim_params)))
        
        s += "\n" + "  Standard Error [relative]: " + str(tuple(map(format_x, self.SE))) +\
              ", [" + str(tuple(map(format_x, self.SE_relative))) + "]"
        
        s += "\n" + "  Pseudo R^2: " + str(format_x(self.pseudo_r_squared))
        
        s += "\n" + "  BIC [relative]: " + str(format_x(self.BIC)) +\
              ", [" + str(format_x(self.BIC_relative)) + "]"
        s += "\n" + "="*50
        
        if string:
            return s
        
        print(s)
    
    
    def null_loglike(self, epsilon=1e-10):
        return self.loglike((1.+epsilon, 0.0))
    
    def pseudo_r_squared(self, params):
        return 1-self.loglike(params)/self.null_loglike()
    
    
    def predict(self, params, ranks=None, freqs=True, n_obs=None, 
                correct_for_finite_domain=True):
        if ranks is None:
            ranks = self.exog
        ranks = np.asarray(ranks)
        
        if n_obs is None:
            n_obs = self.n_obs
            
        alpha, beta = params
        pred_probs = self.prob(params, ranks=ranks, log=False)
        
        if correct_for_finite_domain:
            if not freqs:
                raise NotImplementedError("Correction for "\
                                          "finite domain not implemented with probabilities!")
            return pred_probs*(n_obs/np.sum(pred_probs))
        
        if freqs:
            return n_obs*pred_probs
        
        return pred_probs



In [15]:
# Returns a dataframe containing the mean frequencies and ranks, as well as 
# the estimated frequencies from Zipf's law and the error between the (log) mean
# frequencies and (log) estimated frequencies.
def zipfs_law(df):
    mandelbrot = Mandelbrot(df['Frequency'], df['Rank'])
    mandelbrot_fit = mandelbrot.fit(start_params=np.asarray([1.0, 1.0]), # [1.0, 1.0]
                                method="powell", full_output=True, disp=0)
    mandelbrot.register_fit(mandelbrot_fit)
    mandelbrot.print_result()
    
    model_params = mandelbrot.optim_params
    alpha, beta =  mandelbrot.optim_params
    preds = mandelbrot.predict(model_params, df['Rank'])
    df['Estimated frequency'] = preds
    df['Rank (log)'] = np.log(df['Rank'])
    df['Frequency (log)'] = np.log(df['Frequency'])
    df['Estimated frequency (log)'] = np.log(df['Estimated frequency'])
    df['Error'] = df['Frequency (log)'] - df['Estimated frequency (log)']
    return mandelbrot, df

In [16]:
def zipf_entropy(alpha, dx=1e-10):
    if alpha <= 1.0:
        raise ValueError("Entropy undefined for the given parameter:\n" + 
                         str(alpha))
    return alpha*(-derivative(zeta, alpha, dx=dx))/zeta(alpha) + lg(zeta(alpha))

def mandelbrot_entropy(alpha, beta, dx=1e-10):
    if alpha <= 1.0 or beta <= 1.0:
        raise ValueError("Entropy undefined for the given parameters:\n" + 
                         str(alpha) + " and " + str(beta))
    zeta_b = lambda a: zeta(a, beta+1)
    return alpha*(-derivative(zeta_b, alpha, dx=dx))/zeta_b(alpha) + lg(zeta_b(alpha))


def neg_log_likelihood(zipf_model, ranks, freqs):
    mle_params = zipf_model.optim_params
    log_rank_probs = zipf_model.prob(params=mle_params, ranks=ranks, log=True)    
    return -freqs*log_rank_probs
    
    
def empirical_entropy(zipf_model, joint_rank_freqs):
    rs = list(joint_rank_freqs["Rank"])
    fs = list(joint_rank_freqs["Frequency"])
    ranks = np.asarray(rs)
    freqs = np.asarray(fs)
    n = np.sum(freqs)
    return (1/n)*np.sum(neg_log_likelihood(zipf_model, ranks, freqs))

def typicality(zipf_model, joint_rank_freqs):
    mle_params = zipf_model.optim_params
    return mandelbrot_entropy(*mle_params) - empirical_entropy(zipf_model, joint_rank_freqs)



In [18]:
with open("1000000_26_8.txt" , encoding="utf8") as handle:
    test =  [l.strip() for l in handle.readlines()]


sep_corps = [make_file(corpus, multi = True) for corpus in test]

sep_corps1 = []
for x in sep_corps:
    sep_corps1.append(x[0])
len(sep_corps1)

k = 400000
m = 6

rank_corpora, freq_corpora = subsampling(sep_corps1, k=k, m=m)
ranks_freqs_df = ranks_freqs(rank_corpora, freq_corpora)
# Hieruit krijg je dus een dataframe met voor elk woord de rank, frequency,
# estimated frequency volgens Zipf's law en de error.
mand, df = zipfs_law(ranks_freqs_df)

typicality(mand, ranks_freqs_df)

MANDELBROT
  Optimal Parameters (1.14, 3.92)
  Standard Error [relative]: (0.000341, 0.0242), [(0.0003, 0.00617)]
  Pseudo R^2: 0.653
  BIC [relative]: 3680000.0, [2.88]


6.166909563683695

In [22]:
with open("1000000_2_0.txt" , encoding="utf8") as handle:
    test =  [l.strip() for l in handle.readlines()]


sep_corps = [make_file(corpus, multi = True) for corpus in test]

sep_corps1 = []
for x in sep_corps:
    sep_corps1.append(x[0])
len(sep_corps1)

k = 400000
m = 6

rank_corpora, freq_corpora = subsampling(sep_corps1, k=k, m=m)
ranks_freqs_df = ranks_freqs(rank_corpora, freq_corpora)
# Hieruit krijg je dus een dataframe met voor elk woord de rank, frequency,
# estimated frequency volgens Zipf's law en de error.
mand, df = zipfs_law(ranks_freqs_df)

typicality(mand, ranks_freqs_df)

MANDELBROT
  Optimal Parameters (1.19, 3.06)
  Standard Error [relative]: (0.000503, 0.0265), [(0.000423, 0.00866)]
  Pseudo R^2: 0.708
  BIC [relative]: 2900000.0, [3.43]


4.824719672035206

In [24]:
with open("1000000_10_8.txt" , encoding="utf8") as handle:
    test =  [l.strip() for l in handle.readlines()]


sep_corps = [make_file(corpus, multi = True) for corpus in test]

sep_corps1 = []
for x in sep_corps:
    sep_corps1.append(x[0])
len(sep_corps1)

k = 400000
m = 6

rank_corpora, freq_corpora = subsampling(sep_corps1, k=k, m=m)
ranks_freqs_df = ranks_freqs(rank_corpora, freq_corpora)
# Hieruit krijg je dus een dataframe met voor elk woord de rank, frequency,
# estimated frequency volgens Zipf's law en de error.
mand, df = zipfs_law(ranks_freqs_df)

typicality(mand, ranks_freqs_df)

MANDELBROT
  Optimal Parameters (1.18, 3.17)
  Standard Error [relative]: (0.000463, 0.0267), [(0.000394, 0.00841)]
  Pseudo R^2: 0.696
  BIC [relative]: 3060000.0, [3.29]


5.055302335405379

In [25]:
with open("1000000_22_3.txt" , encoding="utf8") as handle:
    test =  [l.strip() for l in handle.readlines()]


sep_corps = [make_file(corpus, multi = True) for corpus in test]

sep_corps1 = []
for x in sep_corps:
    sep_corps1.append(x[0])
len(sep_corps1)

k = 400000
m = 6

rank_corpora, freq_corpora = subsampling(sep_corps1, k=k, m=m)
ranks_freqs_df = ranks_freqs(rank_corpora, freq_corpora)
# Hieruit krijg je dus een dataframe met voor elk woord de rank, frequency,
# estimated frequency volgens Zipf's law en de error.
mand, df = zipfs_law(ranks_freqs_df)

typicality(mand, ranks_freqs_df)

MANDELBROT
  Optimal Parameters (1.15, 3.58)
  Standard Error [relative]: (0.00038, 0.0259), [(0.000331, 0.00723)]
  Pseudo R^2: 0.667
  BIC [relative]: 3460000.0, [3.01]


5.708401525222168